In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

In [3]:
#use this to download file from the URL
train_file_path = tf.keras.utils.get_file('train.csv', TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file('test.csv', TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [5]:
#Make numpy values easier to read
np.set_printoptions(precision=3, suppress=True)

# Load data
To start, let's look at the top of the CSV file to see how it is formatted.

You can load this using pandas, and pass the NumPy arrays to TensorFlow. If you need to scale up to a large set of files, or need a loader that integrates with TensorFlow and tf.data then use the tf.data.experimental.make_csv_dataset function:

The only column you need to identify explicitly is the one with the value that the model is intended to predict.

In [9]:
LABEL_COLUMN = 'survived'
LABELS = [0,1]

In [10]:
#Now read the CSV data from the file and create a dataset.
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5, #Artificially small to make ex easy to show
        label_name=LABEL_COLUMN,
        na_value='?',
        num_epochs=1,
        ignore_errors=True,
        **kwargs)
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [11]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key,value.numpy()))

Each item in the dataset is a batch, represented as a tuple of (many examples, many labels). The data from the examples is organized in column-based tensors (rather than row-based tensors), each with as many elements as the batch size (5 in this case).

It might help to see this yourself.

In [12]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [28. 36. 28. 28. 18.]
n_siblings_spouses  : [0 0 0 1 0]
parch               : [0 1 0 1 0]
fare                : [  7.896 512.329   7.225  15.246  73.5  ]
class               : [b'Third' b'First' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'B' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Cherbourg' b'Cherbourg' b'Cherbourg' b'Southampton']
alone               : [b'y' b'n' b'y' b'n' b'y']


As you can see, the columns in the CSV are named. The dataset constructor will pick these names up automatically. If the file you are working with does not contain the column names in the first line, pass them in a list of strings to the column_names argument in the make_csv_dataset function.

In [13]:
CSV_COLUMNS  = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'female' b'female' b'female' b'female']
age                 : [39. 26. 34. 18. 37.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 0 1 1 0]
fare                : [24.15  78.85  23.     9.35   9.587]
class               : [b'Third' b'First' b'Second' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'n' b'n' b'y']


# Data preprocessing
A CSV file can contain a variety of data types. Typically you want to convert from those mixed types to a fixed length vector before feeding the data into your model.

TensorFlow has a built-in system for describing common input conversions: tf.feature_column

You can preprocess your data using any tool you like (like nltk or sklearn), and just pass the processed output to TensorFlow.

The primary advantage of doing the preprocessing inside your model is that when you export the model it includes the preprocessing. This way you can pass the raw data directly to your model.

### Continuous data
If your data is already in an appropriate numeric format, you can pack the data into a vector before passing it off to the model:

In [14]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path,
                          select_columns = SELECT_COLUMNS,
                          column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [ 1. 33. 23. 24. 28.]
n_siblings_spouses  : [1. 0. 0. 0. 0.]
parch               : [2. 0. 0. 0. 0.]
fare                : [20.575  9.5   10.5   13.    13.   ]


In [15]:
#you can iterate through the dataset
example_batch, labels_batch = next(iter(temp_dataset))

In [16]:
example_batch, labels_batch

(OrderedDict([('age',
               <tf.Tensor: shape=(5,), dtype=float32, numpy=array([20. , 14. , 14.5, 19. , 28. ], dtype=float32)>),
              ('n_siblings_spouses',
               <tf.Tensor: shape=(5,), dtype=float32, numpy=array([1., 0., 1., 0., 0.], dtype=float32)>),
              ('parch',
               <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>),
              ('fare',
               <tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 7.925,  7.854, 14.454, 30.   ,  8.113], dtype=float32)>)]),
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 0, 0, 1, 1], dtype=int32)>)

In [17]:
#simple function to pack together all the columns
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [18]:
#aply this to each element of the dataset
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[18.     0.     0.     8.3  ]
 [59.     0.     0.    13.5  ]
 [19.     0.     0.    10.171]
 [26.     0.     0.    18.788]
 [23.     0.     0.     7.925]]

[0 0 0 1 0]


If you have mixed datatypes you may want to separate out these simple-numeric fields. The tf.feature_column api can handle them, but this incurs some overhead and should be avoided unless really necessary. Switch back to the mixed dataset:

In [19]:
show_batch(raw_train_data)

sex                 : [b'female' b'female' b'male' b'female' b'male']
age                 : [19. 31. 11. 36. 28.]
n_siblings_spouses  : [1 0 1 1 0]
parch               : [0 2 2 0 0]
fare                : [ 26.    164.867 120.     26.      7.896]
class               : [b'Second' b'First' b'First' b'Second' b'Third']
deck                : [b'unknown' b'C' b'B' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'n' b'n' b'n' b'y']


In [20]:
example_batch, labels_batch = next(iter(temp_dataset))

In [22]:
#so define a more general preprocessor that selects a list of 
#numeric features and packs them into a single column

class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names
        
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat,tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        
        return features, labels

In [23]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']
packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [24]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'female']
class               : [b'Third' b'Third' b'Third' b'First' b'Second']
deck                : [b'F' b'unknown' b'unknown' b'D' b'unknown']
embark_town         : [b'Queenstown' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'y']
numeric             : [[28.     0.     0.     7.75 ]
 [26.     0.     0.     8.05 ]
 [27.     0.     0.     7.925]
 [21.     0.     1.    77.287]
 [38.     0.     0.    13.   ]]


In [25]:
example_batch, labels_batch = next(iter(packed_train_data))
example_batch, labels_batch

(OrderedDict([('sex',
               <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'male', b'female', b'male', b'male', b'female'], dtype=object)>),
              ('class',
               <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Second', b'First', b'First', b'Third', b'First'], dtype=object)>),
              ('deck',
               <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'unknown', b'unknown', b'E', b'unknown', b'unknown'], dtype=object)>),
              ('embark_town', <tf.Tensor: shape=(5,), dtype=string, numpy=
               array([b'Southampton', b'Southampton', b'Southampton', b'Southampton',
                      b'Cherbourg'], dtype=object)>),
              ('alone',
               <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'n', b'n', b'n', b'n', b'y'], dtype=object)>),
              ('numeric', <tf.Tensor: shape=(5, 4), dtype=float32, numpy=
               array([[  0.83 ,   0.   ,   2.   ,  29.   ],
                      [ 28.   ,   1.  

# Data Normalization
Continuous data should always be normalized.

In [29]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [30]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [33]:
def normalize_numeric_data(data, mean, std):
    return (data-mean)/std

Now create a numeric column. The tf.feature_columns.numeric_column API accepts a normalizer_fn argument, which will be run on each batch.

Bind the MEAN and STD to the normalizer fn using functools.partial

In [36]:
#see what you just created
normalize = functools.partial(normalize_numeric_data,mean=MEAN,std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalize,
                                                  shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x14c825bf8>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

When you train the model, include this feature column to select and center this block of numeric data:

In [37]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[  0.83 ,   0.   ,   2.   ,  29.   ],
       [ 28.   ,   1.   ,   0.   , 133.65 ],
       [ 46.   ,   1.   ,   0.   ,  61.175],
       [ 22.   ,   1.   ,   0.   ,   7.25 ],
       [ 35.   ,   0.   ,   0.   , 512.329]], dtype=float32)>

In [38]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-2.302, -0.474,  2.043, -0.099],
       [-0.13 ,  0.395, -0.479,  1.818],
       [ 1.308,  0.395, -0.479,  0.491],
       [-0.61 ,  0.395, -0.479, -0.497],
       [ 0.429, -0.474, -0.479,  8.754]], dtype=float32)

The mean based normalization used here requires knowing the means of each column ahead of time.

# Categorical data
Some of the columns in the CSV data are categorical columns. That is, the content should be one of a limited set of options.

Use the tf.feature_column API to create a collection with a tf.feature_column.indicator_column for each categorical column.

In [39]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [40]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
                key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [41]:
#see what we created
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [44]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


This will be become part of a data processing input later when you build the model.

# Combined preprocessing layer

Add the two feature column collections and pass them to a tf.keras.layers.DenseFeatures to
create an input layer that will extract and preprocess both input types:

In [45]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     1.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -2.302 -0.474
  2.043 -0.099  1.     0.   ]


# Build the Model
Build a tf.keras.Sequential, starting with the preprocessing_layer

In [50]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics = ['accuracy'])


# Train, evaluate, and predict
Now the model can be instantiated and trained.

In [51]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [52]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 2s 12ms/step - loss: 0.5036 - accuracy: 0.7512
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4260 - accuracy: 0.8182
Epoch 3/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4033 - accuracy: 0.8278
Epoch 4/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3941 - accuracy: 0.8357
Epoch 5/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3876 - accuracy: 0.8309
Epoch 6/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3724 - accuracy: 0.8405
Epoch 7/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3685 - accuracy: 0.8405
Epoch 8/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3590 - accuracy: 0.8437
Epoch 9/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3637 - accuracy: 0.8469
Epoch 10/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3475 - accuracy: 0.848

In [53]:
#Once the model is trained, you can check its accuracy on the test_data set.
test_loss, test_accuracy = model.evaluate(test_data)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

     53/Unknown - 1s 10ms/step - loss: 0.4631 - accuracy: 0.8485

Test Loss 0.46309821448236144, Test Accuracy 0.8484848737716675


In [54]:
# Use tf.keras.Model.predict to infer labels on a batch or a dataset of batches.
predictions = model.predict(test_data)

#show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    print("Predicted survival {:.2%}".format(prediction[0]),
         " | Actual outcome: ",
          ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival -114.27%  | Actual outcome:  DIED
Predicted survival 875.28%  | Actual outcome:  DIED
Predicted survival -226.04%  | Actual outcome:  DIED
Predicted survival -234.12%  | Actual outcome:  SURVIVED
Predicted survival 22.33%  | Actual outcome:  SURVIVED
